## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-01-16-50-34 +0000,nypost,Prep school teacher accused of groping 15-year...,https://nypost.com/2025/09/01/us-news/teacher-...
1,2025-09-01-16-50-31 +0000,nyt,California’s High Potency Cannabis is Fueling ...,https://www.nytimes.com/2025/09/01/world/europ...
2,2025-09-01-16-49-30 +0000,nypost,House lawmakers to meet with Jeffrey Epstein’s...,https://nypost.com/2025/09/01/us-news/house-la...
3,2025-09-01-16-49-00 +0000,wsj,"Hundreds Killed in 6.0-Magnitude Earthquake, A...",https://www.wsj.com/world/asia/afghanistan-str...
4,2025-09-01-16-48-36 +0000,nyt,What to Know About the Earthquake in Afghanistan,https://www.nytimes.com/2025/09/01/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
146,trump,31
39,china,13
31,killed,10
217,new,9
35,afghanistan,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
220,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...,70
203,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...,63
85,2025-09-01-11-00-00 +0000,wsj,"The cash benefits of the new tax law, signed b...",https://www.wsj.com/politics/policy/companies-...,60
226,2025-08-31-17-11-30 +0000,nypost,"Trump says he, Putin and Zelensky will meet to...",https://nypost.com/2025/08/31/world-news/trump...,60
189,2025-08-31-21-45-30 +0000,nypost,Trump mulls ‘GREAT’ plan to pay Palestinians $...,https://nypost.com/2025/08/31/us-news/trump-mu...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
220,70,2025-08-31-18-12-00 +0000,wsj,The Supreme Court has flirted with a flurry of...,https://www.wsj.com/politics/policy/trump-tari...
195,39,2025-08-31-21-23-44 +0000,nypost,Israel begins heavy airstrikes ahead of Gaza C...,https://nypost.com/2025/08/31/world-news/israe...
35,38,2025-09-01-14-47-18 +0000,nypost,Rudy Giuliani to remain in hospital after he w...,https://nypost.com/2025/09/01/us-news/rudy-giu...
10,34,2025-09-01-16-31-09 +0000,bbc,EU chief von der Leyen's plane hit by suspecte...,https://www.bbc.com/news/articles/c9d07z1439zo...
78,32,2025-09-01-11-51-28 +0000,nypost,Mom speaks out on hellish jail cell after bein...,https://nypost.com/2025/09/01/world-news/mom-w...
139,31,2025-09-01-07-50-54 +0000,bbc,BYD shares slide as China's EV price war hits ...,https://www.bbc.com/news/articles/cx2pr1wwq44o...
58,30,2025-09-01-13-53-10 +0000,bbc,BBC reports from scene as Afghanistan earthqua...,https://www.bbc.com/news/videos/c3v3gg6zr3yo?a...
130,28,2025-09-01-08-53-15 +0000,nypost,Rosie O’Donnell apologizes for calling trans M...,https://nypost.com/2025/09/01/us-news/rosie-od...
48,27,2025-09-01-14-18-42 +0000,nypost,Millions of people face flood threat in Great ...,https://nypost.com/2025/09/01/us-news/millions...
203,26,2025-08-31-19-58-46 +0000,nypost,Illinois Gov. Pritzker floats conspiracy theor...,https://nypost.com/2025/08/31/us-news/pritzker...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
